# 完整股票分析流程演示

本Notebook将演示如何使用tushare获取股票数据，计算技术指标，并检测RSI背离信号。

In [ ]:
import sys
sys.path.append('.')

import os
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from data.stock_data import get_stock_universe
from signals.calculator import calculate_all_signals
from signals.annotator import annotate_signals
from visualization.chart import create_candlestick_chart

# 设置tushare token
TUSHARE_TOKEN = os.getenv('TUSHARE_TOKEN')
if not TUSHARE_TOKEN:
    raise ValueError('请设置TUSHARE_TOKEN环境变量')

## 1. 数据获取

从tushare获取指定股票的历史数据。

In [ ]:
# 获取股票数据 (以沪深300指数成分股为例)
universes = ['HS300']
date = '2023-12-01'

stock_data = get_stock_universe(universes, date)
print(f'获取到{len(stock_data["HS300"])}}只股票的数据')

# 选择一只股票进行分析 (以第一只为例子)
selected_stock = list(stock_data['HS300'].keys())[0]
df = stock_data['HS300'][selected_stock]
print(f'选择股票: {selected_stock}')
df.head()

## 2. 技术指标计算

计算RSI、MACD和布林带等技术指标。

In [ ]:
# 计算所有技术指标
signals = calculate_all_signals(df)

# 显示计算得到的指标
print('计算得到的技术指标:')
for key in signals.keys():
    print(f'- {key}')

# 查看RSI值
print(f'
RSI值 (最近5天):')
print(signals['rsi'].tail())

## 3. RSI可视化

绘制RSI指标图表。

In [ ]:
# 绘制RSI图表
fig_rsi = go.Figure()
fig_rsi.add_trace(go.Scatter(x=df.index, y=signals['rsi'], mode='lines', name='RSI'))
fig_rsi.add_hline(y=70, line_dash='dash', line_color='red', annotation_text='超买')
fig_rsi.add_hline(y=30, line_dash='dash', line_color='green', annotation_text='超卖')
fig_rsi.update_layout(title=f'{selected_stock} RSI指标', xaxis_title='日期', yaxis_title='RSI')
fig_rsi.show()

## 4. MACD可视化

绘制MACD指标图表。

In [ ]:
# 绘制MACD图表
fig_macd = go.Figure()
fig_macd.add_trace(go.Scatter(x=df.index, y=signals['macd']['macd_line'], mode='lines', name='MACD'))
fig_macd.add_trace(go.Scatter(x=df.index, y=signals['macd']['signal_line'], mode='lines', name='Signal'))
fig_macd.add_trace(go.Bar(x=df.index, y=signals['macd']['histogram'], name='Histogram'))
fig_macd.update_layout(title=f'{selected_stock} MACD指标', xaxis_title='日期', yaxis_title='MACD')
fig_macd.show()

## 5. 布林带可视化

绘制布林带指标图表。

In [ ]:
# 绘制布林带图表
fig_bb = go.Figure()
fig_bb.add_trace(go.Scatter(x=df.index, y=df['close'], mode='lines', name='收盘价'))
fig_bb.add_trace(go.Scatter(x=df.index, y=signals['bollinger_bands']['upper_band'], mode='lines', name='上轨'))
fig_bb.add_trace(go.Scatter(x=df.index, y=signals['bollinger_bands']['middle_band'], mode='lines', name='中轨'))
fig_bb.add_trace(go.Scatter(x=df.index, y=signals['bollinger_bands']['lower_band'], mode='lines', name='下轨'))
fig_bb.update_layout(title=f'{selected_stock} 布林带指标', xaxis_title='日期', yaxis_title='价格')
fig_bb.show()

## 6. 多指标综合图表

将价格和所有技术指标绘制在同一图表中。

In [ ]:
# 使用项目中的函数创建综合图表
fig_combined = create_candlestick_chart(df, signals)
fig_combined.update_layout(title=f'{selected_stock} 综合技术指标分析')
fig_combined.show()

## 7. RSI背离检测

检测RSI与价格之间的背离信号。

In [ ]:
# 查看RSI背离信号
rsi_divergence = signals['rsi_divergence']

print('RSI背离信号:')
for key in rsi_divergence.keys():
    count = rsi_divergence[key].sum()
    print(f'- {key}: {count}次')

# 标识出出现背离的日期
divergence_dates = []
for key in rsi_divergence.keys():
    divergence_dates.extend(rsi_divergence[key][rsi_divergence[key]].index.tolist())

if divergence_dates:
    print(f'
出现RSI背离的日期:')
    for date in divergence_dates:
        print(f'- {date}')
else:
    print('
未检测到RSI背离信号')

## 8. 流程总结

以上步骤展示了完整的股票技术分析流程：
1. 从tushare获取股票数据
2. 计算各种技术指标
3. 可视化各项指标
4. 检测RSI背离信号

这个流程可以帮助投资者更好地理解股票的价格走势和潜在的买卖信号。